# WRI Global Power Plant Database Ingestion Pipeline (Step 2)
# Trino Get Started Demo Sample: Data Ingestion
Copyright (C) 2021 OS-Climate

This sample shows:
* How to connect to pachyderm service and retrieve data for loading
* How to load data into Trino


Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas, pandas profiling and pyarrow
%pip install pandas
%pip install -U pandas-profiling[notebook]
%pip install pyarrow

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools

# Python client for Pachyderm
%pip install python_pachyderm

In [ ]:
# Report exceptions only in cells output
%xmode Minimal

In [1]:
from dotenv import dotenv_values, load_dotenv
import os
import pathlib
import pandas as pd
import trino
from sqlalchemy.engine import create_engine
import osc_ingest_trino as osc
import python_pachyderm

Define Environment and Execution Variables

In [2]:
# Load environment variables from credentials.env
osc.load_credentials_dotenv()

Connect to Pachyderm service using the Python client, retrieve the data from ingested file and load into a dataframe

In [3]:
# Create a pachyderm client by manually setting up host and port
client = python_pachyderm.Client(os.environ['PACH_ENDPOINT'], os.environ['PACH_PORT'])

In [4]:
# Check for committed data files
print(list(client.list_file(("gppd","master"), "/global_power_plant_database_v_1_3/")))

[file {
  commit {
    branch {
      repo {
        name: "gppd"
        type: "user"
      }
      name: "master"
    }
    id: "47471da1ef9f4f5ba4410fc17887983a"
  }
  path: "/global_power_plant_database_v_1_3/global_power_plant_database.csv"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1650532143
  nanos: 409389000
}
size_bytes: 11973385
hash: "\304|:tF\261\020&\267;\340\217\241U\361;\030le \024\200\021\363R\230t\003\"5\261\363"
, file {
  commit {
    branch {
      repo {
        name: "gppd"
        type: "user"
      }
      name: "master"
    }
    id: "47471da1ef9f4f5ba4410fc17887983a"
  }
  path: "/global_power_plant_database_v_1_3/historic_global_power_plant_database.csv"
  datum: "default"
}
file_type: FILE
committed {
  seconds: 1650532143
  nanos: 409389000
}
size_bytes: 109413187
hash: "eO=\233D\023FOv\326S\263\321\277\362h\244\364\264\371\005{wV\352Yq\312\353\306\372\226"
]


In [5]:
# Verify the csv file can be read
f = client.get_file(("gppd", "master"), "/global_power_plant_database_v_1_3/historic_global_power_plant_database.csv")
df_gppd = pd.read_csv(f).convert_dtypes()

In [6]:
print(df_gppd.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 419232 entries, 0 to 419231
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   country                 419232 non-null  string 
 1   country_long            419232 non-null  string 
 2   name                    419232 non-null  string 
 3   gppd_idnr               419232 non-null  string 
 4   capacity_mw             419232 non-null  Float64
 5   latitude                419232 non-null  Float64
 6   longitude               419232 non-null  Float64
 7   primary_fuel            419232 non-null  string 
 8   other_fuel1             23328 non-null   string 
 9   other_fuel2             3312 non-null    string 
 10  other_fuel3             1104 non-null    string 
 11  commissioning_year      209364 non-null  Float64
 12  owner                   250416 non-null  string 
 13  source                  419052 non-null  string 
 14  url                 

In [7]:
df_gppd.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,primary_fuel,other_fuel1,other_fuel2,...,owner,source,url,geolocation_source,wepp_id,year_of_capacity_data,generation_data_source,value,observation_method,observation_year
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.0,32.322,65.119,Hydro,<NA>,<NA>,...,<NA>,GEODB,http://globalenergyobservatory.org,GEODB,1009793,2017,<NA>,<NA>,generation_gwh,2013
1,AFG,Afghanistan,Kandahar DOG,WKS0070144,10.0,31.67,65.795,Solar,<NA>,<NA>,...,<NA>,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,<NA>,<NA>,<NA>,<NA>,generation_gwh,2013
2,AFG,Afghanistan,Kandahar JOL,WKS0071196,10.0,31.623,65.792,Solar,<NA>,<NA>,...,<NA>,Wiki-Solar,https://www.wiki-solar.org,Wiki-Solar,<NA>,<NA>,<NA>,<NA>,generation_gwh,2013
3,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.0,34.556,69.4787,Hydro,<NA>,<NA>,...,<NA>,GEODB,http://globalenergyobservatory.org,GEODB,1009795,2017,<NA>,<NA>,generation_gwh,2013
4,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.0,34.641,69.717,Hydro,<NA>,<NA>,...,<NA>,GEODB,http://globalenergyobservatory.org,GEODB,1009797,2017,<NA>,<NA>,generation_gwh,2013


Connect to Trino and load the data

In [8]:
# Define execution variables for loading to Iceberg / Trino
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'sandbox'
ingest_table = 'gppd'

In [9]:
engine = osc.attach_trino_engine(verbose=True, catalog=ingest_catalog)

using connect string: trino://caldeirav@trino-secure-odh-trino.apps.odh-cl2.apps.os-climate.org:443/osc_datacommons_dev


In [10]:
# Show available schemas to ensure trino connection is set correctly
schema_show_sql = f"""
show schemas in {ingest_catalog}
"""
schema_show = engine.execute(schema_show_sql)
print(schema_show.fetchall())

[('default',), ('demo_dv',), ('iceberg_demo',), ('information_schema',), ('sandbox',)]


Create target table

In [11]:
# Drop table if exists in case we run the pipeline multiple times
sql=f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

[(True,)]


In [12]:
# Create table definition and create table query
columnschema = osc.create_table_schema_pairs(df_gppd)
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['country']
)
"""
print(tabledef)


create table if not exists osc_datacommons_dev.sandbox.gppd(
    country varchar,
    country_long varchar,
    name varchar,
    gppd_idnr varchar,
    capacity_mw double,
    latitude double,
    longitude double,
    primary_fuel varchar,
    other_fuel1 varchar,
    other_fuel2 varchar,
    other_fuel3 varchar,
    commissioning_year double,
    owner varchar,
    source varchar,
    url varchar,
    geolocation_source varchar,
    wepp_id varchar,
    year_of_capacity_data bigint,
    generation_data_source varchar,
    value double,
    observation_method varchar,
    observation_year bigint
) with (
    format = 'ORC',
    partitioning = array['country']
)



In [13]:
# Execute table creation if the table does not exist
qres = engine.execute(tabledef)
print(qres.fetchall())

[(True,)]


In [14]:
# Replace NaN observations with zero for graphing
df_gppd['value'] = df_gppd['value'].fillna(0)

In [15]:
# Filter the dataframe to one country in order to fasten the demo
df_gppd = df_gppd[df_gppd.isin(['Afghanistan']).any(axis=1)]

In [16]:
# Insert the dataframe data in batches of 100
df_gppd.to_sql(ingest_table,
           con=engine,
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 100, verbose = False))